In [ ]:
def to_base_string(s):
    base_s = ""
    bases = ['A', 'T', 'G', 'C', '_']
    for i in range(len(s)):
        val = s[i]
        base_s += bases[val]
    return base_s

def to_base_list(base_s):
    s = []
    for i in range(len(base_s)):
        b = base_s[i]
        if(b == 'A'):
            s.append(0)
        elif(b == 'T'):
            s.append(1)
        elif(b =='G'):
            s.append(2)
        elif(b =='C'):
            s.append(3)
        elif(b=='_'):
            s.append(4)
        else:
            raise Exception(f"Invalid base string, contains character {b}.")
    return s

def tbl(s):
    return to_base_list(s)

def tbs(l):
    return to_base_string(l)

In [ ]:
def print_find_alignment_many(score,lst,s2,i1,i2):
    print(f"i1:{i1}, i2:{i2}")
    for i in range(len(lst)):
        print(i2 * '+' + lst[i][0] * "_" + tbs(lst[i][1]))
    print(i1 * '+' + tbs(s2))
    print(f"score:{score}")
    print("==========")
    
def show_alignment(s,i):
    pan = 10
    if(i < 0):
        pan = pan + i
    return pan * '+' + i * '_' + to_base_string(s) + f"    :{i}"

In [ ]:
def calc_overlap_many(lst, s2, i1, i2):
    total_overlap = 0
    for i in range(len(lst)):
        index = lst[i][0]
        s1 = lst[i][1]
        
        overlap = min(len(s1) - i1 + index, len(s2) - i2 - index)
        if(overlap > 0):
            total_overlap += overlap
    return total_overlap

def calc_score_basic_many(lst, s2, i1, i2):
       
#     s2 = s2[i1:]
      
    total_score = 0
#     overlap = calc_overlap_many(lst, s2,i1,i2)
    
    while(len(s2) > i2 and get_furthest_index(lst) - 1 > i1):

        for i in range(len(lst)):
            s1 = lst[i][1]
            index = lst[i][0]
            
            if(index <= i2 and len(s1) + index > i1):
                score = 0
                if(s1[i1-index] == s2[i2]):
                    score = 1
                total_score += score
        i1 += 1
        i2 += 1
#     print(total_score, overlap)
    return total_score

In [ ]:
def get_furthest_index(lst):
    return max([i+len(l) for (i,l) in lst])

def align_many(lst,s2, fb = False):
    if len(lst) == 0:
        return 0
       
    i1 = get_furthest_index(lst) -1
    i2 = 0
    
    max_score = -100
    max_s2 = []
    max_i2 = 0
    
    print_i1 = 0
    print_i2 = 0
    
    count = 0
    while(i2 < len(s2)):               
        score = calc_score_basic_many(lst, s2, i1,i2)
            
        if(score > max_score):
            if(fb):                       
                print_find_alignment_many(score,lst,s2,i1,i2)
            
            print_i1 = i1
            print_i2 = i2
            
            max_score = score
            max_s2 = s2
            if(i1 > 0):
                max_i2 = i1
            else:
                max_i2 = -1*i2
        if(i1 > 0):
            i1 -= 1
        else:
            i2 += 1
   
    return max_i2


In [ ]:
#test 1
# lst = [
#     [0,to_base_list("AATTCC")],
#     [1,to_base_list("ATTCCG")],
#     [2,to_base_list("TTCCGG")],
#     [3,to_base_list("TCCGGA")],
#     [4,to_base_list("CCGGAA")]
# ]
# s = tbl("AATTCCGGAA")
# find_alignment_many(lst, s)

#test2
# lst = [
#     [1,to_base_list("AAA")],
#     [1,to_base_list("A")]
# ]
# s = tbl("AA")
# find_alignment_many(lst, s)

#test3
lst = [
    [0,tbl("CTCTTGGGGGATAAGCCTGTTATC")],
    [0,tbl("ACTCTTGGGGATAAGCCTGTTATCA")],
    [1,tbl("CTCTTGGGGATAAGCCTGTTATC")],
    [0,tbl("CTCTTGGGGGATAAGCCTGTTATC")],
    [0,tbl("CTCTTGGGGGATAAGCCTGTTATC")],
    [2,tbl("TCTTGGGGATAAGCCTGTTATCC")],
    [2,tbl("CTTGGGGGATAAGCCTGTTATCCA")],
    [3,tbl("CTTGGGGATAAGCCTGTTATCC")],
    [3,tbl("CTTGGGGATAAGCCTGTTATCCC")],
    [4,tbl("CTTGGGGATAAGCCTGTTATCCC")],
]
s = tbl("TGGGGATAAGCCTGTTATCCC")
# find_alignment_many(lst, s, True)

In [ ]:
path = "/Users/felix/MsC/DNA/mastersthesis/felix/assemblydata.npy" #path to the label / unassembled sequences
import numpy as np

file = np.load(path, allow_pickle=True)
sequences_list = []
sequences = file[0]
reference = file[1]

for i in range(len(sequences)):
    sequences_list.append(to_base_list(sequences[i]))
    


In [ ]:
import collections
alignments = []
max_length = 0

window = 10

seq_list=sequences_list[0:68]

# s1 = seq_list[0]
# s2 = seq_list[1]
# index = align_many([(0, s1)],s2)

# alignments.append([0, s1])
# alignments.append([index,s2])

for i, seq in enumerate(seq_list):
    to_align_sequences = alignments[max(0, i-window):i]
    index = align_many(to_align_sequences, seq)
    
    #debug
#     if(len(to_align_sequences) > 0 and abs(index - to_align_sequences[-1][0]) > 5):
#         align_many(to_align_sequences, seq, fb=True)
#         print("======")
    
    alignments.append((index,seq))

    if(index + len(seq) > max_length):
        max_length = index + len(seq)
    
for i, alignment in enumerate(alignments):
    print(f"{i:03d} - {show_alignment(alignment[1], alignment[0])}")
    

In [ ]:
testseq = [(17, [0, 3, 3, 1, 3, 3, 3, 2, 1, 3, 2, 0, 1, 2, 1, 2, 2, 0, 3, 1, 3, 1, 1, 2, 2]), (16, [0, 2, 3, 3, 1, 3, 3, 3, 2, 1, 3, 2, 0, 1, 2, 1, 2, 2, 0, 3, 1, 3, 1, 1, 2, 2]), (17, [0, 3, 3, 1, 3, 3, 3, 2, 1, 3, 2, 0, 1, 2, 1, 2, 2, 0, 3, 1, 3, 1, 1, 2, 2, 2]), (19, [3, 3, 1, 3, 3, 2, 1, 3, 2, 0, 1, 2, 1, 2, 2, 0, 3, 1, 3, 1, 1, 2, 2, 2]), (18, [3, 3, 1, 3, 3, 3, 2, 1, 3, 2, 0, 1, 2, 1, 2, 2, 0, 3, 1, 3, 1, 1, 2, 2, 2]), (18, [3, 3, 1, 3, 3, 3, 2, 1, 3, 2, 0, 1, 2, 1, 2, 2, 0, 3, 1, 3, 1, 1, 2, 2, 2, 1, 1]), (18, [3, 3, 1, 3, 3, 3, 2, 1, 3, 2, 0, 1, 2, 1, 2, 2, 0, 3, 1, 3, 1, 1, 2, 2, 2, 2, 1]), (20, [1, 3, 3, 3, 2, 1, 3, 2, 0, 1, 2, 1, 2, 2, 0, 3, 1, 3, 1, 1, 2, 2, 2, 2, 2, 1]), (22, [3, 3, 2, 1, 3, 2, 0, 1, 2, 1, 2, 2, 0, 3, 1, 3, 1, 1, 2, 2, 2, 2, 2, 0]), (22, [3, 2, 1, 3, 2, 0, 1, 2, 1, 2, 2, 0, 3, 1, 3, 1, 1, 2, 2, 2, 2, 2])]
testalgn = [3, 2, 1, 3, 2, 0, 1, 1, 2, 2, 0, 3, 1, 3, 1, 1, 2, 2, 2, 2, 2, 0, 1]

In [ ]:
for a,b in testseq:
    print(f"{' '*a}{tbs(b)}")
al = align_many(testseq, testalgn)
print(f"{' '*al}{tbs(testalgn)}")
print(al)

In [ ]:
calc_score_basic_many(testseq, testalgn, 7, 0)